In [23]:
import pandas as pd
import numpy as np
import requests
import json

from google.colab import drive
from io import StringIO

In [ ]:
drive.mount('/content/drive')
config_file_path = "/content/drive/MyDrive/Skrypty/config.json"

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

workouts_summary_input = config['workouts_summary_input']
workouts_summary_output = config['workouts_summary_output']

document_key = workouts_summary_input.split('/')[-2]
url_running = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv&id={document_key}&gid=0'
url_cycling = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv&id={document_key}&gid=525762321'

In [25]:
running_response = requests.get(url_running)
running_response_content = running_response.content.decode('utf-8')
data = StringIO(running_response_content)
df = pd.read_csv(data, header=1)

mapped_data = []
for index, row in df.iterrows():
    for col in df.columns[1:]:
        if pd.notna(row[col]) and col != 'średnia' and col != 'miesiąc / rok' and row['miesiąc / rok'] != 'razem':
            mapped_data.append({
                'rok': col,
                'miesiąc': row['miesiąc / rok'],
                'dystans': row[col]
            })

running_df = pd.DataFrame(mapped_data)
running_df['type'] = 'Running'

In [26]:
cycling_response = requests.get(url_cycling)
cycling_response_content = cycling_response.content.decode('utf-8')
data = StringIO(cycling_response_content)
df = pd.read_csv(data, header=1)

mapped_data = []
for index, row in df.iterrows():
    for col in df.columns[1:]:
        if pd.notna(row[col]) and col != 'średnia' and col != 'miesiąc / rok' and row['miesiąc / rok'] != 'razem':
            mapped_data.append({
                'rok': col,
                'miesiąc': row['miesiąc / rok'],
                'dystans': row[col]
            })

cycling_df = pd.DataFrame(mapped_data)
cycling_df['type'] = 'Cycling'

In [27]:
workouts_df = pd.concat([running_df, cycling_df], ignore_index=True)

month_dict = {
    'styczeń': '01', 'luty': '02', 'marzec': '03', 'kwiecień': '04',
    'maj': '05', 'czerwiec': '06', 'lipiec': '07', 'sierpień': '08',
    'wrzesień': '09', 'październik': '10', 'listopad': '11', 'grudzień': '12'}

workouts_df['month'] = workouts_df['miesiąc'].map(month_dict)
workouts_df['date'] = pd.to_datetime(workouts_df['rok'].astype(str) + '-' + workouts_df['miesiąc'].astype(str), format='%Y-%m', errors='coerce')
workouts_df.sort_values(by=['rok', 'month'], inplace=True)
workouts_df.drop(columns=['date', 'month'], inplace=True)
workouts_df['dystans'].replace('-', np.nan, inplace=True)

In [28]:
workouts_df.to_excel(workouts_summary_output, index=False)